<a id="Shallow_Water_Bathymetry_top"></a>
# Shallow Water Bathymetry
### Visualizing Differences in Depth With Spectral Analysis
<hr>

# Notebook Summary

* Import data from LANDSAT 8
* A bathymetry index is calculated
* Contrast is adjusted to make a more interpretable visualization.
>Citation: [Stumpf, Richard P., Kristine Holderied, and Mark Sinclair. "Determination of water depth with high‐resolution satellite imagery over variable bottom types." Limnology and Oceanography 48.1part2 (2003): 547-556.](https://www.slideshare.net/fernandojeffersonprudencioparedes/stumpf-et-al-2003)
<hr>

# Index  

* [Import Dependencies and Connect to the Data Cube](#Shallow_Water_Bathymetry_import)
* [Choose the Platform and Product](#Shallow_Water_Bathymetry_plat_prod)
* [Define the Extents of the Analysis](#Shallow_Water_Bathymetry_define_extents)
* [Retrieve the Data](#Shallow_Water_Bathymetry_retrieve_data)
* [Calculate the Bathymetry and NDWI Indices](#Shallow_Water_Bathymetry_bathymetry)
* [Export Unmasked GeoTIFF](#Shallow_Water_Bathymetry_export_unmasked)
* [Mask the Dataset Using the Quality Column and NDWI](#Shallow_Water_Bathymetry_mask)
* [Create a Visualization Function](#Shallow_Water_Bathymetry_vis_func)
* [Visualize the Bathymetry](#Shallow_Water_Bathymetry_bath_vis)
* [Visualize the Bathymetry With Adjusted Contrast](#Shallow_Water_Bathymetry_bath_vis_better)

<hr>

# How It Works

Bathymetry is the measurement of depth in bodies of water(Oceans, Seas or Lakes).  This notebook illustrates a technique for deriving depth of shallow water areas using purely optical features from Landsat Collection 1 imagery and draws heavily from the publication [Determination of water depth with high-resolution satelite imagery over variable bottom types](https://www.slideshare.net/fernandojeffersonprudencioparedes/stumpf-et-al-2003).

<br>

**Bathymetry Index**  
  
This bathymetry index uses optical `green` and `blue` values on a logarithmic scale with two tunable coefficients `m0` and `m1`.
  

$$ BATH =  m_0*\frac{ln(blue)}{ln(green)} -m_1$$  

Where: 
- `m0` is a tunable scaling factor to tune the ratio to depth <br>
- `m1` is the offset for a depth of 0 meters.

<br>
<div class="alert-info"><br>
<b>Note: </b> that for our purposes, $m_0$ and $m_1$ are equal to <b>1</b> and <b>0</b> respectively, since we cannot determine the baseline nor the offset from spectral reflectance alone. This effectively simplifies the formula to: $$\frac{ln(blue)}{ln(green)}$$
<br>

</div>


#### Bathymetry Index Function

In [ ]:
import sys
import os
sys.path.append(os.environ.get('NOTEBOOK_ROOT'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def bathymetry_index(df, m0 = 1, m1 = 0):
    return m0*(np.log(df.blue)/np.log(df.green))+m1

## <span id="Shallow_Water_Bathymetry_import">Import Dependencies and Connect to the Data Cube [&#9652;](#Shallow_Water_Bathymetry_top)</span>  

In [ ]:
import datacube
dc = datacube.Datacube()

## <span id="Shallow_Water_Bathymetry_plat_prod">Choose the Platform and Product [&#9652;](#Shallow_Water_Bathymetry_top)</span>

In [ ]:
#List the products available on this server/device
dc.list_products()

In [ ]:
#create a list of the desired platforms
platform = "LANDSAT_8"
product = "ls8_level1_usgs"

## <span id="Shallow_Water_Bathymetry_define_extents">Define the Extents of the Analysis [&#9652;](#Shallow_Water_Bathymetry_top)</span>

### Region bounds

In [ ]:
# East Coast of Australia
lat_subsect = (-31.7, -32.2)
lon_subsect = (152.4, 152.9)

In [ ]:
print('''
Latitude:\t{0}\t\tRange:\t{2} degrees
Longitude:\t{1}\t\tRange:\t{3} degrees
'''.format(lat_subsect,
           lon_subsect,
           max(lat_subsect)-min(lat_subsect),
           max(lon_subsect)-min(lon_subsect)))

### Display

In [ ]:
from utils.data_cube_utilities.dc_display_map import display_map      
display_map(latitude = lat_subsect,longitude = lon_subsect)

## <span id="Shallow_Water_Bathymetry_retrieve_data">Retrieve the Data [&#9652;](#Shallow_Water_Bathymetry_top)</span>

#### Load and integrate datasets

In [ ]:
%%time
ds = dc.load(lat = lat_subsect,
             lon = lon_subsect,
             platform = platform,
             product = product,
             output_crs = "EPSG:32756",
             measurements = ["red","blue","green","nir","quality"],
             resolution = (-30,30))

In [ ]:
ds

#### Preview the Data

In [ ]:
from utils.data_cube_utilities.dc_rgb import rgb
rgb(ds.isel(time=6), x_coord='x', y_coord='y')
plt.show()

## <span id="Shallow_Water_Bathymetry_bathymetry">Calculate the Bathymetry and NDWI Indices [&#9652;](#Shallow_Water_Bathymetry_top)</span>
> * The bathymetry function is located at the top of this notebook.

In [ ]:
# Create Bathemtry Index column
ds["bathymetry"] = bathymetry_index(ds)

In [ ]:
from utils.data_cube_utilities.dc_water_classifier import NDWI
# (green - nir) / (green + nir)
ds["ndwi"] = NDWI(ds, band_pair=1)

<hr>

#### Preview Combined Dataset

In [ ]:
ds

## <span id="Shallow_Water_Bathymetry_export_unmasked">Export Unmasked GeoTIFF [&#9652;](#Shallow_Water_Bathymetry_top)</span>

In [ ]:
import os
from utils.data_cube_utilities.import_export import export_xarray_to_multiple_geotiffs

unmasked_dir = "geotiffs/landsat8/unmasked"
if not os.path.exists(unmasked_dir):
    os.makedirs(unmasked_dir)
export_xarray_to_multiple_geotiffs(ds, unmasked_dir + "/unmasked.tif",
                                   x_coord='x', y_coord='y')

## <span id="Shallow_Water_Bathymetry_mask">Mask the Dataset Using the Quality Column and NDWI [&#9652;](#Shallow_Water_Bathymetry_top)</span>

In [ ]:
# preview values
np.unique(ds["quality"])

#### Use NDWI to Mask Out Land
> The threshold can be tuned if need be to better fit the RGB image above. <br>
> Unfortunately our existing WOFS algorithm is designed to work with Surface Reflectance (SR) and does not work with this data yet but with a few modifications it could be made to do so.  We will approximate the WOFs mask with `NDWI` for now.

In [ ]:
# Tunable threshold for masking the land out
threshold = .05

water = (ds.ndwi>threshold).values

In [ ]:
#preview one time slice to determine the effectiveness of the NDWI masking
rgb(ds.where(water).isel(time=6), x_coord='x', y_coord='y')
plt.show()

In [ ]:
from utils.data_cube_utilities.dc_mosaic import ls8_oli_unpack_qa
clear_xarray  = ls8_oli_unpack_qa(ds.quality, "clear")

In [ ]:
full_mask = np.logical_and(clear_xarray, water)

ds = ds.where(full_mask)

## <span id="Shallow_Water_Bathymetry_vis_func">Create a Visualization Function [&#9652;](#Shallow_Water_Bathymetry_top)</span>

#### Visualize the distribution of the bathymetry index for the water pixels

In [ ]:
plt.figure(figsize=[15,5])

#Visualize the distribution of the remaining data
sns.boxplot(ds['bathymetry'])
plt.show()

> <b>Interpretation: </b> We can see that most of the values fall within a very short range.  We can scale our plot's cmap limits to fit the specific quantile ranges for the bathymetry index so we can achieve better contrast from our plots.

In [ ]:
#set the quantile range in either direction from the median value
def get_quantile_range(col, quantile_range = .25):
    low = ds[col].quantile(.5 - quantile_range,["time","y","x"]).values
    high = ds[col].quantile(.5 + quantile_range,["time","y","x"]).values
    return low,high

In [ ]:
#Custom function for a color mapping object
from matplotlib.colors import LinearSegmentedColormap

def custom_color_mapper(name = "custom", val_range = (1.96,1.96), colors = "RdGnBu"):
    custom_cmap = LinearSegmentedColormap.from_list(name,colors=colors)
    
    min, max = val_range
    step = max/10.0
    Z = [min,0],[0,max]
    levels = np.arange(min,max+step,step)
    cust_map = plt.contourf(Z, 100, cmap=custom_cmap)
    plt.clf()
    return cust_map.cmap

In [ ]:
def mean_value_visual(ds, col, figsize = [15,15], cmap = "GnBu", low=None, high=None):
    if low is None: low = np.min(ds[col]).values
    if high is None: high = np.max(ds[col]).values
    ds.reduce(np.nanmean,dim=["time"])[col].plot.imshow(figsize = figsize, cmap=cmap,  
                                                        vmin=low, vmax=high)

## <span id="Shallow_Water_Bathymetry_bath_vis">Visualize the Bathymetry [&#9652;](#Shallow_Water_Bathymetry_top)</span>

In [ ]:
mean_value_visual(ds, "bathymetry", cmap="GnBu")

## <span id="Shallow_Water_Bathymetry_bath_vis_better">Visualize the Bathymetry With Adjusted Contrast [&#9652;](#Shallow_Water_Bathymetry_top)</span>

> If we clamp the range of the plot using different quantile ranges we can see relative differences in higher contrast.

In [ ]:
# create range using the 10th and 90th quantile
low, high = get_quantile_range("bathymetry", .40)


custom = custom_color_mapper(val_range=(low,high),
                             colors=["darkred","red","orange","yellow","green",
                                     "blue","darkblue","black"])

mean_value_visual(ds, "bathymetry", cmap=custom, low=low, high=high)

In [ ]:
# create range using the 5th and 95th quantile
low, high = get_quantile_range("bathymetry", .45)


custom = custom_color_mapper(val_range=(low,high),
                             colors=["darkred","red","orange","yellow","green",
                                     "blue","darkblue","black"])

mean_value_visual(ds, "bathymetry", cmap = custom, low=low, high = high)

In [ ]:
# create range using the 2nd and 98th quantile
low, high = get_quantile_range("bathymetry", .48)


custom = custom_color_mapper(val_range=(low,high),
                             colors=["darkred","red","orange","yellow","green",
                                     "blue","darkblue","black"])

mean_value_visual(ds, "bathymetry", cmap=custom, low=low, high=high)

In [ ]:
# create range using the 1st and 99th quantile
low, high = get_quantile_range("bathymetry", .49)


custom = custom_color_mapper(val_range=(low,high),
                             colors=["darkred","red","orange","yellow","green",
                                     "blue","darkblue","black"])

mean_value_visual(ds, "bathymetry", cmap=custom, low=low, high=high)